In [76]:
from PIL import Image
import os

def convert_images_to_jpg(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate over all files in the input folder
    for filename in os.listdir(input_folder):
        # Construct full file path
        file_path = os.path.join(input_folder, filename)
        
        # Check if it is a file
        if os.path.isfile(file_path):
            try:
                # Open the image file
                with Image.open(file_path) as img:
                    # Convert the image to RGB (necessary for JPEG format)
                    rgb_img = img.convert('RGB')
                    
                    # Create new filename with .jpg extension
                    new_filename = os.path.splitext(filename)[0] + '.jpg'
                    new_file_path = os.path.join(output_folder, new_filename)
                    
                    # Save the image in JPG format
                    rgb_img.save(new_file_path, 'JPEG')
                    
                    print(f'Converted {filename} to {new_filename}')
            except Exception as e:
                print(f'Error processing {filename}: {e}')

# Example usage
input_folder = "C:/Users/USER/Desktop/new그립플라이어"  # Replace with the path to your folder containing images
output_folder = "C:/Users/USER/Desktop/그립플라이어200jpg"  # Replace with the path to your output folder

convert_images_to_jpg(input_folder, output_folder)


Converted 화면 캡처 2024-07-22 151858.jpg to 화면 캡처 2024-07-22 151858.jpg
Converted 화면 캡처 2024-07-22 151913.jpg to 화면 캡처 2024-07-22 151913.jpg
Converted 화면 캡처 2024-07-22 151933.jpg to 화면 캡처 2024-07-22 151933.jpg
Converted 화면 캡처 2024-07-22 151949.jpg to 화면 캡처 2024-07-22 151949.jpg
Converted 화면 캡처 2024-07-22 152005.jpg to 화면 캡처 2024-07-22 152005.jpg
Converted 화면 캡처 2024-07-22 152025.jpg to 화면 캡처 2024-07-22 152025.jpg
Converted 화면 캡처 2024-07-22 152041.jpg to 화면 캡처 2024-07-22 152041.jpg
Converted 화면 캡처 2024-07-22 152059.jpg to 화면 캡처 2024-07-22 152059.jpg
Converted 화면 캡처 2024-07-22 152113.jpg to 화면 캡처 2024-07-22 152113.jpg
Converted 화면 캡처 2024-07-22 152132.jpg to 화면 캡처 2024-07-22 152132.jpg
Converted 화면 캡처 2024-07-22 152143.jpg to 화면 캡처 2024-07-22 152143.jpg
Converted 화면 캡처 2024-07-22 152156.jpg to 화면 캡처 2024-07-22 152156.jpg
Converted 화면 캡처 2024-07-22 152207.jpg to 화면 캡처 2024-07-22 152207.jpg
Converted 화면 캡처 2024-07-22 152221.jpg to 화면 캡처 2024-07-22 152221.jpg
Converted 화면 캡처 2024-07-22 152232.

In [77]:
import os
import re

# 디렉토리 경로 설정
directory = "C:/Users/USER/Desktop/그립플라이어200jpg"

# 디렉토리 내 파일 목록 가져오기
files = os.listdir(directory)

# 정규 표현식으로 파일 이름 정렬 (숫자가 포함된 파일 이름을 기준으로 정렬)
files.sort(key=lambda x: int(re.search(r'\d+', x).group()))

# 파일 이름을 snap_{i+1} 형식으로 변경
for i, filename in enumerate(files):
    # 파일 확장자 추출
    file_extension = os.path.splitext(filename)[1]
    
    # 새 파일 이름 생성
    new_name = f'grip_{i+1}{file_extension}'
    
    # 경로 설정
    old_path = os.path.join(directory, filename)
    new_path = os.path.join(directory, new_name)
    
    # 파일 이름 변경
    os.rename(old_path, new_path)

print("파일 이름 변경 완료")


파일 이름 변경 완료


In [78]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import os, time, uuid

In [79]:
ENDPOINT = os.environ["VISION_TRAINING_ENDPOINT"]
training_key = os.environ["VISION_TRAINING_KEY"]
prediction_key = os.environ["VISION_PREDICTION_KEY"]
prediction_resource_id = os.environ["VISION_PREDICTION_RESOURCE_ID"]

In [80]:
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)

In [81]:
publish_iteration_name = "detectModel"

# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")

# Create a new project
print ("Creating project...")
# Use uuid to avoid project name collisions.
project = trainer.create_project(str(uuid.uuid4()), domain_id=obj_detection_domain.id)

Creating project...


In [82]:
# Make two tags in the new project
snap_tag = trainer.create_tag(project.id, "snap200")
grip_tag = trainer.create_tag(project.id, "grip200")

In [83]:
snap_image_regions = {f"snap_{i+1}": [0, 0, 1, 1] for i in range(500)}
grip_image_regions = {f"grip_{i+1}": [0, 0, 1, 1] for i in range(50)}

In [84]:
base_image_location = "C:/Users/USER/Desktop/그립플라이어200jpg"
# Go through the data table above and create the images
print ("Adding images...")
tagged_images_with_regions = []#태깅을위한 리스트 초기화

Adding images...


In [85]:
for file_name in grip_image_regions.keys():
    x,y,w,h = grip_image_regions[file_name]
    regions = [ Region(tag_id=grip_tag.id, left=x,top=y,width=w,height=h) ]

# 이미지 파일 경로를 구성
    file_path = os.path.join(base_image_location, file_name + ".jpg")
    
    with open(file_path, mode="rb") as image_contents:
        tagged_images_with_regions.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), regions=regions))


In [86]:
# 이미지 리스트를 64개씩 분할
batches = [tagged_images_with_regions[i:i + 64] for i in range(0, len(tagged_images_with_regions), 64)]

for batch in batches:
    upload_result = trainer.create_images_from_files(project.id, ImageFileCreateBatch(images=batch))
    if not upload_result.is_batch_successful:
        print("Image batch upload failed.")
        for image in upload_result.images:
            print("Image status: ", image.status)
        exit(-1)
